In [1]:
import os 
import torch
import torchaudio
import torch.nn as nn
from types import SimpleNamespace

from IPython.display import Audio
import matplotlib.pyplot as plt 
from timm import create_model, list_models
import lightning as L

from gdsc.data import AudioDataset, DataModule
from gdsc.trainer import TrainModule
from gdsc.net import SampleNet

# %load_ext autoreload
# %autoreload 2

C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# list_models()

In [3]:
cfg = SimpleNamespace()

# paths
# cfg.data_folder = ''
# cfg.name = 
# cfg.data_dir = 
# cfg.train_data_folder = cfg.data_dir + "train_audio/"
# cfg.val_data_folder = cfg.data_dir + "train_audio/"
# cfg.output_dir = 

# dataset
# cfg.dataset = "base_ds"
# cfg.min_rating = 0
# cfg.val_df = None
# cfg.batch_size_val = 1
# cfg.train_aug = None
# cfg.val_aug = None
# cfg.test_augs = None
# cfg.wav_len_val = 5  # seconds
# cfg.min_rating = 2.0
# cfg.wav_crop_len = 30  # seconds
# audio
cfg.img_height = 256
cfg.img_weight = 512
cfg.window_size = 2048
cfg.sample_rate = 32000
cfg.fmin = 16
cfg.fmax = 16386
cfg.power = 2
cfg.mel_bins = cfg.img_height
cfg.hop_size = 512 # int(cfg.sample_rate * cfg.wav_crop_len / (cfg.img_weight - 1))
cfg.top_db = 80.0

# img model

cfg.pretrained = False
cfg.pretrained_weights = None
cfg.train = True
cfg.val = False
cfg.in_chans = 1

cfg.alpha = 1
cfg.eval_epochs = 1
cfg.eval_train_epochs = 1
cfg.warmup = 0

cfg.mel_norm = False

cfg.label_smoothing = 0

cfg.remove_pretrained = []

# training
cfg.seed = 123
cfg.save_val_data = True

# ressources
# cfg.mixed_precision = True
# cfg.gpu = 0
# cfg.num_workers = 4 # 18
# cfg.drop_last = True 

cfg.n_classes = 8


# training 
cfg.lr = 0.001
cfg.epochs = 5
cfg.batch_size = 1
cfg.batch_size_val = 1
cfg.backbone = "resnet34"

In [4]:
dm = DataModule(data_dir=r'C:\Users\dolemm\OneDrive - Capgemini\Documents\work\train_audio', batch_size=1)
model = SampleNet(cfg)
tmod = TrainModule(model, loss_fn=nn.BCEWithLogitsLoss(reduction="none"), optimizer_name='Adam', optimizer_hparams={"lr": cfg.lr, "weight_decay": 1e-4})

C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(


In [5]:
trainer = L.Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
)
trainer.fit(tmod, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name    | Type              | Params
----------------------------------------------
0 | model   | SampleNet         | 21.3 M
1 | loss_fn | BCEWithLogitsLoss | 0     
----------------------------------------------
21.3 M    Trainable params
0         Non-train

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\dolemm\AppData\Local\anaconda3\envs\GDSC23\lib\site-packages\lightning\pytorch\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 7/7 [00:04<00:00,  1.71it/s, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 7/7 [00:03<00:00,  1.82it/s, v_num=9]       
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 7/7 [00:03<00:00,  1.81it/s, v_num=9]       
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 7/7 [00:03<00:00,  1.79it/s, v_num=9]       
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 7/7 [00:03<00:00,  1.83it/s, v_num=9]       
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 7/7 [03:47<00:00, 32.52s/it, v_num=9]       

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 7/7 [07:29<00:00, 64.21s/it, v_num=9]


In [6]:
%debug

> c:\users\dolemm\appdata\local\anaconda3\envs\gdsc23\lib\site-packages\lightning\pytorch\core\module.py(617)__to_tensor()
    615         )
    616         if not torch.numel(value) == 1:
--> 617             raise ValueError(
    618                 f"`self.log({name}, {value})` was called, but the tensor must have a single element."
    619                 f" You can try doing `self.log({name}, {value}.mean())`"



ipdb>  up


> c:\users\dolemm\appdata\local\anaconda3\envs\gdsc23\lib\site-packages\lightning_utilities\core\apply_func.py(51)apply_to_collection()
     49     # Breaking condition
     50     if isinstance(data, dtype) and (wrong_dtype is None or not isinstance(data, wrong_dtype)):
---> 51         return function(data, *args, **kwargs)
     52 
     53     elem_type = type(data)



ipdb>  up


> c:\users\dolemm\appdata\local\anaconda3\envs\gdsc23\lib\site-packages\lightning\pytorch\core\module.py(444)log()
    442             )
    443 
--> 444         value = apply_to_collection(value, (Tensor, numbers.Number), self.__to_tensor, name)
    445 
    446         if trainer._logger_connector.should_reset_tensors(self._current_fx_name):



...

ipdb>  preds.sigmoid()


tensor([[0.5232, 0.4368, 0.5125, 0.6221, 0.5822, 0.5343, 0.5222, 0.4309]],
       grad_fn=<SigmoidBackward0>)


ipdb>  loss.mean(dim=1).sum()


tensor(0.7315, grad_fn=<SumBackward0>)


ipdb>  exit


In [33]:
dm.val.__getitem__(1)

(tensor([[[-1.2418e-06,  8.5255e-06, -1.2656e-05,  ...,  1.1560e-05,
            4.1606e-05,  5.7300e-06]]]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.]]))